# *k*-NN DTW
Nearest Neighbour on time-series data using Dynamic Time Warping (DTW).  
Using the `tslearn` library.  
https://tslearn.readthedocs.io/  
1. How DTW works.
2. Using DTW to identify nearest neighbours.
3. *k*-NN time-series classification using DTW. 

### How DTW works
Calculating the DTW mapping for two time-series. 

In [ ]:
import numpy
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn import metrics

In [ ]:
x = numpy.array([[7,8,10,10,9,8,7,4,2,1,2,4,7,12,13,9,7],
                [7,7,8,9,10,10,8,7,4,3,6,12,12,9,7,7,8]])
dataset = x.reshape(2,17,1)

In [ ]:
import matplotlib.pyplot as plt

ax = plt.axes()        
ax.xaxis.grid(which = 'both')
ax.set_ylabel('Amplitude')
ax.set_xlabel('Time')
#plt.grid(b = True, which='both')
plt.plot(x[0], linewidth = 3, color = 'r')
plt.plot(x[1], linewidth = 3, color = 'b')
plt.minorticks_on()

plt.show()

In [ ]:
scaler = TimeSeriesScalerMeanVariance()  # Rescale time series
dataset_scaled = scaler.fit_transform(dataset)

Plotting the DTW mapping.   

In [ ]:
# Based on code by Romain Tavenard
# License: BSD 3 clause

# the length of the time-series
sz = dataset_scaled.shape[1]

path, sim = metrics.dtw_path(dataset_scaled[0], dataset_scaled[1])

plt.figure(1, figsize=(8, 8))

# definitions for the axes
left, bottom = 0.01, 0.1
w_ts = h_ts = 0.2
left_h = left + w_ts + 0.02
width = height = 0.65
bottom_h = bottom + height + 0.02

rect_s_y = [left, bottom, w_ts, height]
rect_gram = [left_h, bottom, width, height]
rect_s_x = [left_h, bottom_h, width, h_ts]

ax_gram = plt.axes(rect_gram)
ax_s_x = plt.axes(rect_s_x)
ax_s_y = plt.axes(rect_s_y)

mat = cdist(dataset_scaled[0], dataset_scaled[1])

ax_gram.imshow(mat)
ax_gram.axis("off")
ax_gram.autoscale(False)
ax_gram.plot([j for (i, j) in path], [i for (i, j) in path], "w-", linewidth=3.)

ax_s_x.plot(numpy.arange(sz), dataset_scaled[1], "b-", linewidth=3.)
ax_s_x.axis("off")
ax_s_x.set_xlim((0, sz - 1))

ax_s_y.plot(- dataset_scaled[0], numpy.arange(sz)[::-1], "r-", linewidth=3.)
ax_s_y.axis("off")
ax_s_y.set_ylim((0, sz - 1))

plt.show()

The similarity and the mapping path for the two time-series.

In [ ]:
print(sim)
print(path)

## Find Nearest Neighbours using DTW
We use `KNeighborsTimeSeries` to find the nearest neighbours using DTW.  
We plot the query and the nearest and farthest neighbours. 

In [ ]:
from tslearn.neighbors import KNeighborsTimeSeries
from tslearn.datasets import UCR_UEA_datasets
uea_ucr = UCR_UEA_datasets()

In [ ]:
X_train, y_train, X_test, y_test = uea_ucr.load_dataset('TwoLeadECG')
X_train.shape,X_test.shape

`n_neighbors` is set to the size of the training set so  
`knn` will return all the training data sorted by similarity to the query. 

In [ ]:
knn = KNeighborsTimeSeries(n_neighbors=len(X_train))
knn.fit(X_train)

In [ ]:
 q = X_test[1].reshape(1,82,1)

In [ ]:
ret = knn.kneighbors(q)
nn = ret[1][0][0]    # The nearest neighbour
fn = ret[1][0][-1]   # The farthest neighbor
nn,fn

In [ ]:
plt.plot(q[0], linewidth = 2, color = 'r')
plt.plot(X_train[nn],linewidth = 2, color = 'b')
plt.title('The Nearest Neighbour')

In [ ]:
plt.plot(q[0], linewidth = 2, color = 'r')
plt.plot(X_train[fn],linewidth = 2, color = 'g')
plt.title('The Farthest Neighbour')

## Time Series Classification using DTW
Compare the performance of DTW with Euclidean distance. 

In [ ]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.metrics import accuracy_score

In [ ]:
tsc_euc = KNeighborsTimeSeriesClassifier(n_neighbors=2, metric="euclidean")
tsc_dtw = KNeighborsTimeSeriesClassifier(n_neighbors=2, metric="dtw", n_jobs = -1)

In [ ]:
tsc_euc.fit(X_train, y_train)
tsc_dtw.fit(X_train, y_train)

In [ ]:
y_euc = tsc_euc.predict(X_test)
y_dtw = tsc_dtw.predict(X_test)

In [ ]:
print("Accuracy using Euclidean Distance: {0:.2f}".format(accuracy_score(y_euc,y_test))) 
print("Accuracy using DTW: {0:.2f}".format(accuracy_score(y_dtw,y_test))) 